In [6]:
import numpy as np

%config Completer.use_jedi = False

np.seterr(divide="ignore", invalid="ignore")

from numpy import linalg as LA
import pandas as pd
import matplotlib.pyplot as plt

#%matplotlib notebook
#%load_ext lab_black

import scipy
import scipy.stats
from scipy import fftpack, stats
from scipy.interpolate import make_interp_spline, BSpline
from scipy.optimize import curve_fit

from skimage import color
from skimage.transform import hough_circle, hough_circle_peaks
from skimage.feature import canny
from skimage.draw import circle_perimeter
from skimage.util import img_as_ubyte, invert
from skimage.measure import profile_line

from IPython.display import display, HTML, Markdown, Latex, clear_output
import ipywidgets as widgets

# from pybloqs import *

from highcharts import Highchart

import sys
import os
import fnmatch
from itertools import compress

# reload(sys)

import math
import glob2
import pydicom as dc
from pydicom import dcmread
from pydicom.filebase import DicomBytesIO

In [7]:
import ipywidgets as widgets

In [8]:
uploader=widgets.FileUpload(
    accept='',  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
    multiple=True  # True to accept multiple files upload else False
)

In [9]:
display(uploader)

FileUpload(value={}, description='Upload', multiple=True)

In [10]:
# Intervalos del Catphan 504
CTP528 = np.array([0, -30])  # Resolución espacial
CTP401 = np.array([-35, -45])  # Espesor de corte
CTP515 = np.array([-60, -96])  # Bajo contraste
CTP486 = np.array([-96, -200])  # Homogeneidad

# Intervalos del Catphan 604
CTP732 = np.array([50, 30])  # Resolución espacial
CTP730 = np.array([10, -10])  # Espesor de corte
CTP729 = np.array([-20, -110])  # Bajo contraste y homogeneidad


Catphan_ExtRadius = 100  # radio externo del catphan


In [11]:
def smooth(x, window_len=0, window="hanning"):
    """smooth the data using a window with requested size.
    
    This method is based on the convolution of a scaled window with the signal.
    The signal is prepared by introducing reflected copies of the signal 
    (with the window size) in both ends so that transient parts are minimized
    in the begining and end part of the output signal.
    
    input:
        x: the input signal 
        window_len: the dimension of the smoothing window; should be an odd integer
        window: the type of window from 'flat', 'hanning', 'hamming', 'bartlett', 'blackman'
            flat window will produce a moving average smoothing.

    output:
        the smoothed signal
        
    example:

    t=linspace(-2,2,0.1)
    x=sin(t)+randn(len(t))*0.1
    y=smooth(x)
    
    see also: 
    
    numpy.hanning, numpy.hamming, numpy.bartlett, numpy.blackman, numpy.convolve
    scipy.signal.lfilter
 
    TODO: the window parameter could be the window itself if an array instead of a string
    NOTE: length(output) != length(input), to correct this: return y[(window_len/2-1):-(window_len/2)] instead of just y.
    """

    if x.ndim != 1:
        raise ValueError("smooth only accepts 1 dimension arrays")

    if x.size < window_len:
        raise ValueError("Input vector needs to be bigger than window size")

    if window_len < 3:
        return x

    if not window in ["flat", "hanning", "hamming", "bartlett", "blackman"]:
        raise ValueError(
            "Window is on of 'flat', 'hanning', 'hamming', 'bartlett', 'blackman'"
        )

    s = np.r_[x[window_len - 1 : 0 : -1], x, x[-2 : -window_len - 1 : -1]]
    # print(len(s))
    if window == "flat":  # moving average
        w = np.ones(window_len, "d")
    else:
        w = eval("np." + window + "(window_len)")

    y = np.convolve(w / w.sum(), s, mode="valid")
    return y


def BusquedaCirculos(image, HoughParameters, PixelSize):
    edges = canny(
        image,
        sigma=HoughParameters[0],
        low_threshold=HoughParameters[1],
        high_threshold=HoughParameters[2],
    )
    hough_radii = np.array(HoughParameters[3])
    hough_radii = (hough_radii / PixelSize).astype(np.int)
    hough_res = hough_circle(edges, hough_radii)
    return hough_circle_peaks(
        hough_res, hough_radii, total_num_peaks=HoughParameters[4]
    )


def createCircularMask(h, w, center=None, radius=None):

    if center is None:  # use the middle of the image
        center = [int(w / 2), int(h / 2)]
    if radius is None:  # use the smallest distance between the center and image walls
        radius = min(center[0], center[1], w - center[0], h - center[1])

    Y, X = np.ogrid[:h, :w]
    dist_from_center = np.sqrt((X - center[0]) ** 2 + (Y - center[1]) ** 2)

    mask = dist_from_center <= radius
    return mask


def EndPointProfile(Inicio, Angulo, Radio, image):
    X, Y = Inicio
    return (
        int(X - Radio * math.sin(math.radians(Angulo))),
        int(Y - Radio * math.cos(math.radians(Angulo))),
        image[
            int(X - Radio * math.sin(math.radians(Angulo))),
            int(Y - Radio * math.cos(math.radians(Angulo))),
        ],
    )


def NormalizeMTF(Matrix, Agua, Aire):
    return (Matrix - Aire) / (Agua - Aire)




class Point:
    def __init__(self, x_init, y_init):
        self.x = x_init
        self.y = y_init

    def shiftX(self, d):
        return Point(self.x + d, self.y)

    def shiftY(self, d):
        return Point(self.x, self.y + d)

    def __repr__(self):
        return "".join(["Point(", str(self.x), ",", str(self.y), ")"])

    def __add__(a, b):
        return Point(a.x + b.x, a.y + b.y)

    def __getitem__(self, i):
        return Point(self.x[i], self.y[i])




# TC018
TC018_HoughParameters = [3, 0, 50, [99, 100], 1]
TC018_Limits = [87.5, 112.5]
TC018_OverResolution = 0.1
TC018_MaxAngle = 6
TC018_StepAngle = 0.5
TC018_Angles = [0, 30, 60, 90, 120, 150, 180]

In [12]:
for i in uploader.value.keys():
    print(i)

Img188.dcm
Img189.dcm
Img190.dcm
Img191.dcm
Img192.dcm
Img193.dcm
Img194.dcm
Img195.dcm
Img196.dcm
Img197.dcm
Img198.dcm
Img199.dcm
Img200.dcm
Img201.dcm


In [13]:
def LoadDicomFiles(X):
    output=[]
    for dicom in X.value.keys():
        output.append(dcmread(DicomBytesIO(uploader.value[dicom]['content'])))
    return output

In [16]:
TC018_image=LoadDicomFiles(uploader)

In [17]:
#files=glob2.glob("AXIAL/Uniformidad/*.dcm",recursive=False)

In [18]:
#TC018_image = [dc.read_file(A) for A in files]

| Código| Parámetro&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; | &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Clasificación        
| :- |:- | :-
|[TC018](https://drive.google.com/open?id=14adFusPK1sqlF9hLEqKl86HMZ03XSFzf&disco=AAAADH081mY)| Resolución espacial| ESENCIAL

In [19]:
#####################################################
# TC018_image = datos[Tabla_Resumen[Tabla_Resumen["Z"] == 20].index[0]]
# TC018_image_Res = datos[Tabla_Resumen[Tabla_Resumen["Z"] == -10].index[0]]
#####################################################

In [20]:
TC018_Resolution = TC018_image[0].PixelSpacing[0]
TC018_accums, TC018_cx, TC018_cy, TC018_radii = BusquedaCirculos(
    TC018_image[0].pixel_array, TC018_HoughParameters, TC018_Resolution
)

TC018_cx, TC018_cy

TC018_DistanceMap = np.arange(
    TC018_Limits[0] / TC018_Resolution,
    TC018_Limits[1] / TC018_Resolution,
    TC018_OverResolution,
)

In [21]:
TC018_WaterMask = createCircularMask(
    TC018_image[0].pixel_array.shape[0],
    TC018_image[0].pixel_array.shape[1],
    center=(TC018_cx[0], TC018_cy[0]),
    radius=int(Catphan_ExtRadius * 0.9 / TC018_Resolution)
    * np.logical_not(
        createCircularMask(
            TC018_image[0].pixel_array.shape[0],
            TC018_image[0].pixel_array.shape[1],
            center=(TC018_cx[0], TC018_cy[0]),
            radius=int(Catphan_ExtRadius * 0.8 / TC018_Resolution),
        )
    ),
)
TC018_AirMask = createCircularMask(
    TC018_image[0].pixel_array.shape[0],
    TC018_image[0].pixel_array.shape[1],
    center=(TC018_cx[0], TC018_cy[0]),
    radius=int(Catphan_ExtRadius * 1.3 / TC018_Resolution)
    * np.logical_not(
        createCircularMask(
            TC018_image[0].pixel_array.shape[0],
            TC018_image[0].pixel_array.shape[1],
            center=(TC018_cx[0], TC018_cy[0]),
            radius=int(Catphan_ExtRadius * 1.1 / TC018_Resolution),
        )
    ),
)

HUAgua = TC018_WaterMask * TC018_image[0].pixel_array

HUAire = TC018_AirMask * TC018_image[0].pixel_array


TC018_Angulo = math.sqrt(2 * TC018_Resolution / Catphan_ExtRadius)  # En radianes
TC018_FrecuencyResolution = (
    0.01 / TC018_Resolution
)  # Resolución en frecuencia tomada según IEC62220-1
TC018_FrecuenciaNyquist = 1 / 2.0 / TC018_Resolution
TC018_Oversampling = int(1 / math.tan(TC018_Angulo))  # factor para el "overfitting"
TC018_NumeroDePuntos = int(100 * TC018_Oversampling)

In [22]:
TC018_comp = (
    math.pi
    / 2
    / TC018_FrecuenciaNyquist
    * np.arange(0, TC018_FrecuenciaNyquist, TC018_FrecuencyResolution)
) / np.sin(
    math.pi
    / 2
    / TC018_FrecuenciaNyquist
    * np.arange(0, TC018_FrecuenciaNyquist, TC018_FrecuencyResolution)
)

TC018_image_PA = np.array([X.pixel_array for X in TC018_image])

In [23]:
TC018_Limites = {
    ("Y", "Sup", "Min"): np.max(
        [0, TC018_cy[0] - int(Catphan_ExtRadius / TC018_Resolution) - 50]
    ),
    ("Y", "Inf", "Max"): np.min(
        [
            TC018_image_PA.shape[0] - 1,
            TC018_cy[0] + int(Catphan_ExtRadius / TC018_Resolution) + 50,
        ]
    ),
    ("X", "Izq", "Min"): np.max(
        [0, TC018_cx[0] - int(Catphan_ExtRadius / TC018_Resolution) - 50]
    ),
    ("X", "Dch", "Max"): np.min(
        [
            TC018_image_PA.shape[1] - 1,
            TC018_cx[0] + int(Catphan_ExtRadius / TC018_Resolution) + 50,
        ]
    ),
}

In [24]:
TC018_Data_Y = np.array(
    [
        [
            X[
                TC018_Limites["X", "Izq", "Min"] : TC018_Limites["X", "Izq", "Min"]
                + 100,
                TC018_cy[0] : TC018_cy[0] + TC018_Oversampling,
            ],
            X[
                TC018_Limites["X", "Izq", "Min"] : TC018_Limites["X", "Izq", "Min"]
                + 100,
                TC018_cy[0] - TC018_Oversampling : TC018_cy[0],
            ],
            X[
                TC018_Limites["X", "Dch", "Max"]
                - 100 : TC018_Limites["X", "Dch", "Max"],
                TC018_cy[0] : TC018_cy[0] + TC018_Oversampling,
            ],
            X[
                TC018_Limites["X", "Dch", "Max"]
                - 100 : TC018_Limites["X", "Dch", "Max"],
                TC018_cy[0] - TC018_Oversampling : TC018_cy[0],
            ],
        ]
        for X in TC018_image_PA
    ]
)

In [25]:
# for X in TC018_Data_Y:
# X[1] = np.flip(X[1], axis=0)
# X[3] = np.flip(X[3], axis=0)

In [26]:
TC018_Data_X = np.array(
    [
        [
            X[
                TC018_Limites["X", "Izq", "Min"] : TC018_Limites["X", "Izq", "Min"]
                + 100,
                TC018_cy[0] : TC018_cy[0] + TC018_Oversampling,
            ],
            X[
                TC018_Limites["X", "Izq", "Min"] : TC018_Limites["X", "Izq", "Min"]
                + 100,
                TC018_cy[0] - TC018_Oversampling : TC018_cy[0],
            ],
            X[
                TC018_Limites["X", "Dch", "Max"]
                - 100 : TC018_Limites["X", "Dch", "Max"],
                TC018_cy[0] : TC018_cy[0] + TC018_Oversampling,
            ],
            X[
                TC018_Limites["X", "Dch", "Max"]
                - 100 : TC018_Limites["X", "Dch", "Max"],
                TC018_cy[0] - TC018_Oversampling : TC018_cy[0],
            ],
        ]
        for X in TC018_image_PA
    ]
)

for X in TC018_Data_X:
    X[0] = np.flip(X[0], axis=0)
    X[3] = np.flip(X[3], axis=0)

In [27]:
TC018_Data_Y1 = np.reshape(
    TC018_Data_Y,
    (
        TC018_Data_Y.shape[0] * TC018_Data_Y.shape[1],
        TC018_Data_Y.shape[2] * TC018_Data_Y.shape[3],
    ),
)

In [28]:
TC018_Data_X1 = np.reshape(
    TC018_Data_X,
    (
        TC018_Data_X.shape[0] * TC018_Data_X.shape[1],
        TC018_Data_X.shape[2] * TC018_Data_X.shape[3],
    ),
)

In [29]:
TC018_Data_X1 = np.array(
    [np.reshape(X, (X.shape[0] * X.shape[1])) for Y in TC018_Data_X for X in Y]
)

In [30]:
TC018_Data_Y1 = [
    NormalizeMTF(X, HUAgua[HUAgua > 0.01].mean(), HUAire[HUAire > 0.01].mean())
    for X in TC018_Data_Y1
]

In [31]:
TC018_Data_X1 = [
    NormalizeMTF(X, HUAgua[HUAgua > 0.01].mean(), HUAire[HUAire > 0.01].mean())
    for X in TC018_Data_X1
]

In [32]:
def GetMTF(X):
    return np.absolute(fftpack.fft(np.gradient(X)))

In [33]:
TC018_MTFY = np.array([GetMTF(X) for X in TC018_Data_Y1])
TC018_MTFX = np.array([GetMTF(X) for X in TC018_Data_X1])

In [34]:
def Suavizado(MTF):
    MTF1 = [(MTF[i - 1] + MTF[i] + MTF[i + 1]) / 3 for i in range(1, 49)]
    MTF1.insert(0, 1)
    MTF1.insert(49, (MTF[48] + MTF[49]) / 2)
    MTF1 = np.array(MTF1)
    return MTF1


TC018_MTFY = np.array([Suavizado(X) for X in TC018_MTFY])
TC018_MTFX = np.array([Suavizado(X) for X in TC018_MTFX])

In [35]:
TC018_MTFX_Data = np.around(
    np.vstack([np.arange(0, 10, 0.2), TC018_MTFX.mean(axis=0)[0:50]]).transpose(), 4
)
TC018_MTFY_Data = np.around(
    np.vstack([np.arange(0, 10, 0.2), TC018_MTFY.mean(axis=0)[0:50]]).transpose(), 4
)

In [36]:
def TC018_CalcularMTF(X, data):
    return np.array(
        [
            np.interp(x, np.sort(data.mean(axis=0)[0:50]), np.arange(10, 0, -0.2))
            for x in X
        ]
    )

In [37]:
TC018_SemaforosX = np.around(
    TC018_CalcularMTF(np.array([0.5, 0.1, 0.02]), TC018_MTFX), 2
)
TC018_SemaforosY = np.around(
    TC018_CalcularMTF(np.array([0.5, 0.1, 0.02]), TC018_MTFY), 2
)


def round_up_to_odd(f):
    return (np.ceil(f * 10) // 2 * 2 - 2) / 10

In [38]:
TC018_SemaforosX = round_up_to_odd(TC018_SemaforosX)
TC018_SemaforosY = round_up_to_odd(TC018_SemaforosY)

In [39]:
dataX = [{"x": X[0], "y": X[1], "marker": {"radius": 1}} for X in TC018_MTFX_Data]
dataY = [{"x": X[0], "y": X[1], "marker": {"radius": 1}} for X in TC018_MTFY_Data]
data = [
    {"x": X[0], "y": X[1], "marker": {"radius": 1}}
    for X in (TC018_MTFY_Data + TC018_MTFX_Data) / 2
]

In [40]:
for X in range(len(data)):
    rojo = True
    amarillo = True
    verde = True
    if data[X]["x"] == TC018_SemaforosX[0]:
        data[X]["marker"] = {
            "symbol": "url(https://drive.google.com/uc?id=1ipxqP5SoT96eU-wR_4rUD0JsII4EQChh)"
        }
    elif data[X]["x"] == TC018_SemaforosX[1]:
        data[X]["marker"] = {
            # "symbol": "url(http://localhost:8888/tree/Resources/Yellow.png)"
            "symbol": "url(https://drive.google.com/uc?id=1t7QSLwkE_FOqbzvDVwdzo9wQjdBkMDi2)"
        }
    elif data[X]["x"] == TC018_SemaforosX[2]:
        data[X]["marker"] = {
            # "symbol": "url(http://localhost:8888/tree/Resources/Red.png)"
            "symbol": "url(https://drive.google.com/uc?id=1YH9JlmKBBGZAiS8PET-PZLcBZFCZzLoZ)"
        }

In [41]:
def TC018_OpcionesGraf():
    options = {
        "chart": {"zoomType": "xy"},
        "title": {"text": "MTF"},
        "xAxis": {
            "labels": {
                "useHTML": True,
                "format": "{value} cm<sup>-1</sup>",
                "enabled": True,
            },
            "title": {
                "text": "Frecuencia espacial",
                "style": {"color": "Highcharts.getOptions().colors[1]"},
            },
        },
        "yAxis": [
            {
                "labels": {
                    "format": "{value}",
                    "style": {
                        "color": "Highcharts.getOptions().colors[1]",
                        "step": 150,
                    },
                },
                "title": {
                    "text": "Función de trasferencia de modulación",
                    "style": {"color": "Highcharts.getOptions().colors[1]"},
                },
            }
        ],
        "tooltip": {"shared": True, "valueDecimals": 1},
    }

    return options


TC018_H = Highchart(width=800, height=600)
TC018_H.set_dict_options(TC018_OpcionesGraf())
TC018_H.add_data_set(
    dataX,
    "spline",
    "MTFX",
    xAxis=0,
    yAxis=0,
    tooltip={
        "headerFormat": "frecuencia: {point.x:.2f} cm <sup>-1</sup><br>",
        "pointFormat": '<span style="font-weight: bold; color: {series.color}">{series.name}</span>: <b>{point.y:.4f}</b><br>',
    },
)
TC018_H.add_data_set(
    dataY,
    "spline",
    "MTFY",
    xAxis=0,
    yAxis=0,
    tooltip={
        "headerFormat": "frecuencia: {point.x:.2f} cm <sup>-1</sup><br>",
        "pointFormat": '<span style="font-weight: bold; color: {series.color}">{series.name}</span>: <b>{point.y:.4f}</b><br>',
    },
)
TC018_H.add_data_set(
    data,
    "spline",
    "MTF",
    xAxis=0,
    yAxis=0,
    tooltip={
        "headerFormat": "frecuencia: {point.x:.2f} cm <sup>-1</sup><br>",
        "pointFormat": '<span style="font-weight: bold; color: {series.color}">{series.name}</span>: <b>{point.y:.4f}</b>',
    },
)

display(TC018_H)

In [42]:
pd.DataFrame([[TC018_SemaforosX[0],TC018_SemaforosX[1],TC018_SemaforosX[2]]],columns=['0.5','0.1','0.02'])

,0.5,0.1,0.02
0,4.0,7.6,9.8
